Chapter 10 Dynamic Programming<br>

The term dynamic programming refers to an approach to writing algorithms in which a problem is solved using solutions to the same problem on smaller instances.<br>
In this chapter we take a perticular example problem called "giving change using the fewest coins" and see how it can be optimised using dynamic programming.

10.1 A Greedy Algorithm

In [1]:
def greedyMC(coinValueList, change):
    coinValueList.sort()
    coinValueList.reverse()
    numcoins = 0
    for c in coinValueList:
        numcoins += change // c
        change = change % c
    return numcoins

print(greedyMC([1,5,10,25], 63))
print(greedyMC([1, 21, 25], 63))
print(greedyMC([1, 5, 21, 25], 63))

6
15
7


A first attempt at such a solution might be to assign the change in a greedy way. In that approach, you would simply try to add the largest coin you can until you are done.<br>
Clearly, this does not work. The second example returns 15 and the third returns 7. In both cases, the right answer should have been 3 as can be achieved by returning three 21 cent coins. The problem is not a bug in the code, but an incorrect algorithm. The greedy solution does not work here. Let’s try recursion instead.

10.2 A Recursive Algorithm

In [2]:
def recMC(coinValueList, change):
    minCoins = change
    if change in coinValueList:
        return 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 +recMC(coinValueList, change-i)
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins

print(recMC([1, 21, 25],63))
print(recMC([1, 5, 21, 25],63))

3
3


This works, but it’s very slow.

10.3 A Memoized Version

In [3]:
def memoMC(coinValueList, change, knowResults):
    minCoins = change
    if change in coinValueList:
        knowResults[change] = 1
        return 1
    elif change in knowResults:
        return knowResults[change]
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + memoMC(coinValueList, change-i, knowResults)
            if numCoins < minCoins:
                minCoins = numCoins
                knowResults[change] = minCoins
    return minCoins

print(memoMC([1,5,10,25],63, {}))
knownresults = {}
print(memoMC([1, 5, 10, 21, 25], 63, knownresults))
print(knownresults)

6
3
{25: 1, 26: 2, 21: 1, 10: 1, 11: 2, 5: 1, 6: 2, 1: 1, 12: 3, 7: 3, 13: 4, 8: 4, 14: 5, 9: 5, 15: 2, 16: 3, 27: 3, 22: 2, 17: 4, 28: 4, 23: 3, 18: 5, 29: 5, 24: 4, 19: 6, 30: 2, 20: 2, 31: 2, 32: 3, 33: 4, 34: 5, 35: 2, 36: 3, 37: 4, 38: 5, 39: 6, 40: 3, 41: 3, 42: 2, 43: 3, 44: 4, 45: 3, 46: 2, 47: 3, 48: 4, 49: 5, 50: 2, 51: 3, 52: 3, 53: 4, 54: 5, 55: 3, 56: 3, 57: 4, 58: 5, 59: 6, 60: 3, 61: 4, 62: 4, 63: 3}


This is much faster and now can work with much larger instances.

10.4 A Dynamic Programming Algorithm

In [4]:
def dpMakeChange(coinValueList, change):
    minCoins = [None]*(change + 1)
    for cents in range(change+1):
        minCoins[cents] = cents
        for c in coinValueList:
            if cents >= c:
                minCoins[cents] = min(  minCoins[cents], minCoins[cents - c]+1)
    return minCoins[change]

print(dpMakeChange([1,5,10,21,25], 63))
print(dpMakeChange([1,5,10,21,25], 64))

3
4


Dynamic program builds the results from the bottom up, while the recursive version works top down,

10.5 Another Example<br>

The next problem we’ll solve with dynamic programming is called the **longest common subsequence (LCS)** problem. A subsequence of a string s is a string t such that the characters of t all appear in s in the same order. For example ’abc’ is a subsequence of ’xxxxxaxxxbxxxcxxxx’ The input to the LCS problem is a pair of strings, we’ll call them X and Y. The output is the longest string that is a subsequence of both X and Y.<br>

HINT:<br>
If X and Y end in the same character, then that character is the last character in the longest common subsequence. That is, if X[-1] == Y[-1] then the LCS(X,Y) is LCS(X[:-1], Y[:-1]) + X[-1]. On the other hand, if X[-1] != Y[-1], then at least one of X[-1] or Y[-1] is not in the LCS. In that case, LCS(X,Y) is the longer of LCS(X[:-1],Y) and LCS(X,Y[:-1]). We could turn this into a very tidy recursive algorithm.

In [5]:
def reclcs(X, Y):
    if X == "" or Y =="":
        return ""
    if X[-1] == Y[-1]:
        return reclcs(X[:-1], Y[:-1]) + X[-1]
    else:
        return max([reclcs(X[:-1], Y), reclcs(X, Y[:-1])], key=len)

This takes a long time too.<br>
Let's add memorization

In [6]:
def lcs(X, Y):
    t = {}
    for i in range(len(X)+1): t[(i, 0)] = ""
    for j in range(len(Y)+1): t[(0, j)] = ""

    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            if x == y:
                t[(i+1, j+1)] = t[(i, j)] + x
            else:
                t[(i+1, j+1)] = max([t[(i, j+1)], t[(i+1, j)]], key=len)
    return t[(len(x), len(Y))]

The initialization takes linear time and the main pair of loops iterate O(n2 ) times. The inner loop takes time proportional to the LCS in the worst case, because we will concatenate strings of that length. The total running time is O(kn2 ) where k is the length of the output.